In [ ]:
import pandas as pd
import numpy as np
import os
import re
import string
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset, concatenate_datasets, DatasetDict, load_metric

In [ ]:
# Load Data
CNN_SUM = '/content/dataset_cnn_summarized.xlsx'
KOMPAS_SUM = '/content/dataset_kompas_summarized.xlsx'
TEMPO_SUM = '/content/dataset_tempo_summarized.xlsx'
TURNBACKHOAX_SUM = '/content/dataset_turnbackhoax_summarized.xlsx'

In [ ]:
def excel_to_df(excel):
    df = pd.read_excel(excel, index_col='index')
#   df = df.drop(columns=df.columns[0])
    return df

In [ ]:
df_cnn = excel_to_df(CNN_SUM)
df_kompas = excel_to_df(KOMPAS_SUM)
df_tempo = excel_to_df(TEMPO_SUM)
df_turnbackhoax = excel_to_df(TURNBACKHOAX_SUM)

df_cnn.head(2)

,title,raw timestamp,original,tags,author,url,cleaned,label,timestamp,cleaned token length,summarized
index,,,,,,,,,,,
0,Anies di Milad BKMT: Pengajian Menghasilkan Ib...,"Selasa, 21 Feb 2023 21:22 WIB","Jakarta, CNN Indonesia -- Mantan Gubernur DKI ...",anies baswedan;pengajian;pilpres 2024;badan ko...,CNN Indonesia,https://www.cnnindonesia.com/nasional/20230221...,Anies di Milad BKMT: Pengajian Menghasilkan Ib...,0,2023-02-21 21:22:00,401,Mantan Gubernur DKI Jakarta Anies Baswedan men...
1,Edy Soal Pilgub Sumut: Kalau yang Maju Abal-ab...,"Selasa, 21 Feb 2023 20:46 WIB","Medan, CNN Indonesia -- Gubernur Sumatera Utar...",edy rahmayadi;pemilu 2024;pilkada 2024,CNN Indonesia,https://www.cnnindonesia.com/nasional/20230221...,Edy Soal Pilgub Sumut: Kalau yang Maju Abal-ab...,0,2023-02-21 20:46:00,410,Gubernur Sumatera Utara Edy Rahmayadi membuka ...


In [ ]:
# Drop NaN in used data
df_cnn = df_cnn[df_cnn['cleaned'].notna()]
df_kompas = df_kompas[df_kompas['cleaned'].notna()]
df_tempo = df_tempo[df_tempo['cleaned'].notna()]
df_turnbackhoax = df_turnbackhoax[df_turnbackhoax['cleaned'].notna()]

In [ ]:
# Make sure the type is string
df_cnn['cleaned'] = df_cnn['cleaned'].astype(str)
df_kompas['cleaned'] = df_kompas['cleaned'].astype(str)
df_tempo['cleaned'] = df_tempo['cleaned'].astype(str)
df_turnbackhoax['cleaned'] = df_turnbackhoax['cleaned'].astype(str)

<ipython-input-7-1a467e939ad2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kompas['cleaned'] = df_kompas['cleaned'].astype(str)
<ipython-input-7-1a467e939ad2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_turnbackhoax['cleaned'] = df_turnbackhoax['cleaned'].astype(str)


In [ ]:
# Give space to punctuation
def space_to_punct(text):
    for punct in string.punctuation:
        text = text.replace(punct, f' {punct} ')
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
df_cnn['cleaned'] = df_cnn['cleaned'].map(lambda x: space_to_punct(x))
df_kompas['cleaned'] = df_kompas['cleaned'].map(lambda x: space_to_punct(x))
df_tempo['cleaned'] = df_tempo['cleaned'].map(lambda x: space_to_punct(x))
df_turnbackhoax['cleaned'] = df_turnbackhoax['cleaned'].map(lambda x: space_to_punct(x))


In [ ]:
df_cnn_cleaned = df_cnn[['cleaned']]
df_cnn_cleaned['labels'] = 0
df_kompas_cleaned = df_kompas[['cleaned']]
df_kompas_cleaned['labels'] = 0
df_tempo_cleaned = df_tempo[['cleaned']]
df_tempo_cleaned['labels'] = 0
df_turnbackhoax_cleaned = df_turnbackhoax[['cleaned']]
df_turnbackhoax_cleaned['labels'] = 1


<ipython-input-10-13a271cfe0fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cnn_cleaned['labels'] = 0
<ipython-input-10-13a271cfe0fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kompas_cleaned['labels'] = 0
<ipython-input-10-13a271cfe0fc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [ ]:
def tokenize_and_create_dataset(df_cleaned, labels):
    tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
    tokenized = tokenizer(df_cleaned['cleaned'].tolist(), padding=True, truncation=True, max_length=512)
    return tf.data.Dataset.from_tensor_slices((
        dict(tokenized),
        labels.values
    ))

In [ ]:
tf_ds_cnn = tokenize_and_create_dataset(df_cnn_cleaned, df_cnn_cleaned['labels'])
tf_ds_kompas = tokenize_and_create_dataset(df_kompas_cleaned, df_kompas_cleaned['labels'])
tf_ds_tempo = tokenize_and_create_dataset(df_tempo_cleaned, df_tempo_cleaned['labels'])
tf_ds_turnbackhoax = tokenize_and_create_dataset(df_turnbackhoax_cleaned, df_turnbackhoax_cleaned['labels'])

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Split into train and test
train_size = 0.8

tf_ds_cnn_train, tf_ds_cnn_test = tf_ds_cnn.take(int(len(tf_ds_cnn) * train_size)), tf_ds_cnn.skip(int(len(tf_ds_cnn) * train_size))
tf_ds_kompas_train, tf_ds_kompas_test = tf_ds_kompas.take(int(len(tf_ds_kompas) * train_size)), tf_ds_kompas.skip(int(len(tf_ds_kompas) * train_size))
tf_ds_tempo_train, tf_ds_tempo_test = tf_ds_tempo.take(int(len(tf_ds_tempo) * train_size)), tf_ds_tempo.skip(int(len(tf_ds_tempo) * train_size))
tf_ds_turnbackhoax_train, tf_ds_turnbackhoax_test = tf_ds_turnbackhoax.take(int(len(tf_ds_turnbackhoax) * train_size)), tf_ds_turnbackhoax.skip(int(len(tf_ds_turnbackhoax) * train_size))


In [ ]:
# Concatenate Datasets
tf_ds_train = tf.data.Dataset.concatenate(tf_ds_cnn_train, tf_ds_kompas_train)
tf_ds_train = tf.data.Dataset.concatenate(tf_ds_train, tf_ds_tempo_train)
tf_ds_train = tf.data.Dataset.concatenate(tf_ds_train, tf_ds_turnbackhoax_train)

tf_ds_eval = tf.data.Dataset.concatenate(tf_ds_cnn_test, tf_ds_kompas_test)
tf_ds_eval = tf.data.Dataset.concatenate(tf_ds_eval, tf_ds_tempo_test)
tf_ds_eval = tf.data.Dataset.concatenate(tf_ds_eval, tf_ds_turnbackhoax_test)


In [ ]:
# Model Configuration
model = TFAutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=2)

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Compile Model
model.compile(
    optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Training
batch_size = 8
epochs = 5

model.fit(
    tf_ds_train.shuffle(1000).batch(batch_size),
    epochs=epochs,
    validation_data=tf_ds_eval.batch(batch_size),
    steps_per_epoch=20,
)

Epoch 1/5
20/20 [==============================] - 307s 14s/step - loss: 0.0774 - accuracy: 1.0000 - val_loss: 1.0233 - val_accuracy: 0.7631
Epoch 2/5
20/20 [==============================] - 259s 14s/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 1.1777 - val_accuracy: 0.7631
Epoch 3/5
20/20 [==============================] - 258s 14s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 1.2461 - val_accuracy: 0.7631
Epoch 4/5
20/20 [==============================] - 259s 14s/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.2937 - val_accuracy: 0.7631
Epoch 5/5
20/20 [==============================] - 258s 14s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.3283 - val_accuracy: 0.7631


In [ ]:
# Evaluation
test_loss, test_accuracy = model.evaluate(tf_ds_eval.batch(batch_size))
print(f"Test Accuracy: {test_accuracy}")

687/687 [==============================] - 241s 351ms/step - loss: 1.3283 - accuracy: 0.7631
Test Accuracy: 0.763066828250885


In [ ]:
# Save Model
model.save_weights('indobert_hoax_detection_model.h5')